# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [176]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [177]:
## import csv to dataframe, add try block to deal with directory issues
# csv_import_path = os.path.join('Weather_API','Weather','Export Folder','city_data.csv')
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,54.98,-117.91,valleyview,ca,"(55.066667, -117.283333)",55.066667,-117.283333,30.88,95,99,7.36
1,53.60,-13.37,dingle,ie,"(52.1408333, -10.2688889)",52.140833,-10.268889,52.36,83,100,10.71
2,-32.08,-16.69,jamestown,sh,"(-15.9333333, -5.7166667)",-15.933333,-5.716667,62.64,89,100,23.49
3,37.95,99.76,zhangye,cn,"(38.934167, 100.451667)",38.934167,100.451667,46.98,48,4,16.04
4,49.51,140.54,vanino,ru,"(49.086856, 140.254337)",49.086856,140.254337,44.83,64,24,6.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [178]:
# #configure gmaps with key
# gmaps.configure(api_key=g_key)

In [179]:

# #defines coordinates to plot and weight is humididty
# locations=city_weather_df[['Latitude','Longitude']]
# weighty=city_weather_df['Humidity']

# fig=gmaps.figure(center=(0,0),zoom_level=2)
# heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=100,point_radius=10)
# fig.add_layer(heat)


# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [180]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Temperature']>70)&(city_weather_df['Temperature']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
60,-52.57,48.58,tsihombe,mg,"(-25.3, 45.4833333)",-25.300000,45.483333,72.05,66,0,9.95
81,27.42,79.51,farrukhabad,in,"(27.4, 79.566667)",27.400000,79.566667,79.43,26,0,6.29
134,23.51,1.47,tessalit,ml,"(20.2013889, 1.0088889)",20.201389,1.008889,72.54,20,0,7.11
157,31.12,81.91,dharchula,in,"(29.85, 80.533333)",29.850000,80.533333,71.13,20,0,3.74
216,13.23,58.07,salalah,om,"(17.0175, 54.0827778)",17.017500,54.082778,75.20,78,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [181]:
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

hotel_df.head()
hotel_df['Hotel Name']=""
hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,tsihombe,mg,-25.300000,45.483333,72.05,66,0,9.95,
1,farrukhabad,in,27.400000,79.566667,79.43,26,0,6.29,
2,tessalit,ml,20.201389,1.008889,72.54,20,0,7.11,
3,dharchula,in,29.850000,80.533333,71.13,20,0,3.74,
4,salalah,om,17.017500,54.082778,75.20,78,0,2.24,


In [182]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
x_locations = hotel_df[["Latitude", "Longitude"]]

hotel_df["Hotel Name"]=hotel_info

hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,tsihombe,mg,-25.300000,45.483333,72.05,66,0,9.95,\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><...
1,farrukhabad,in,27.400000,79.566667,79.43,26,0,6.29,\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><...
2,tessalit,ml,20.201389,1.008889,72.54,20,0,7.11,\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><...
3,dharchula,in,29.850000,80.533333,71.13,20,0,3.74,\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><...
4,salalah,om,17.017500,54.082778,75.20,78,0,2.24,\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><...


In [183]:
# marks=gmaps.marker_layer(x_locations)

In [184]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [185]:
# Add marker layer ontop of heat map


# Display figure
